In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
# Import API key
from config import gkey
gmaps.configure(api_key=gkey)

In [2]:
df = pd.read_csv("../Resources/city_Data.csv")

In [3]:
clean_df = df.dropna()

### Humidity Heatmaps

In [53]:
locations = clean_df[['Lat', 'Lng']]
weights = clean_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Filtering Conditions

In [8]:
c1 = clean_df["Max Temp"] <= 20
c2 = clean_df["Max Temp"] >= 10
c3 = clean_df["Humidity"] <= 60
c4 = clean_df["Wind Speed"] > 1
filtered_df = clean_df.loc[c1&c2&c3&c4,:]

In [28]:
filtered_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,busselton,-33.6500,115.3333,12.34,54.0,87.0,12.34,AU,1.663068e+09
11,11,carnarvon,-24.8667,113.6333,19.73,53.0,6.0,19.73,AU,1.663068e+09
64,64,plettenberg bay,-34.0527,23.3716,17.63,56.0,4.0,17.63,ZA,1.663068e+09
88,88,zelenogorskiy,55.0333,87.0000,11.35,49.0,19.0,11.35,RU,1.663068e+09
91,91,lesnoy,57.6198,63.0784,14.93,41.0,69.0,14.93,RU,1.663068e+09
143,143,cradock,-32.1642,25.6192,18.76,25.0,0.0,18.76,ZA,1.663068e+09
151,151,george,-33.9630,22.4617,18.75,52.0,20.0,18.75,ZA,1.663068e+09
161,161,darhan,49.4867,105.9228,15.24,43.0,100.0,15.24,MN,1.663068e+09
178,178,east london,-33.0153,27.9116,19.52,55.0,0.0,19.52,ZA,1.663068e+09
189,189,pagegiai,55.1381,21.9089,17.36,44.0,94.0,17.36,LT,1.663068e+09


In [26]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

row = filtered_df.loc[2,:]
lat = row["Lat"]
lng = row["Lng"]
params = {
    "location": f"{lat},{lng}",
    "keyword": "hotel",
    "radius": "5000",
    "key": gkey
}
req = requests.get(url,params)

In [27]:
pprint.pprint(req.json()["results"][0]["name"])

'Esplanade Hotel'


In [38]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df = pd.DataFrame()

for ind in filtered_df.index:
    lat = filtered_df.loc[ind,"Lat"]
    lng = filtered_df.loc[ind,"Lng"]
    params["location"] = f"{lat},{lng}"
    req = requests.get(url,params)
    try:
        hotel_df.loc[ind,"Hotel Name"] = req.json()["results"][0]["name"]
        hotel_df.loc[ind,"Lat"] = lat
        hotel_df.loc[ind,"Lng"] = lng
        hotel_df.loc[ind,"City"] = filtered_df.loc[ind,"City"]
        hotel_df.loc[ind,"Country"] = filtered_df.loc[ind,"Country"]
    except:
        print("No results, passing")

No results, passing
No results, passing
No results, passing
No results, passing
No results, passing
No results, passing


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,Hotel Name,Lat,Lng,City,Country
2,The Sebel Busselton,-33.6500,115.3333,busselton,AU
11,Carnarvon Hotel,-24.8667,113.6333,carnarvon,AU
64,Bayview Hotel,-34.0527,23.3716,plettenberg bay,ZA
91,Apartamenty Na Sovetskoy,57.6198,63.0784,lesnoy,RU
143,Die Tuishuise and Victoria Manor Cradock,-32.1642,25.6192,cradock,ZA
151,Protea by Marriott Hotel George King George,-33.9630,22.4617,george,ZA
161,MBM HOTEL,49.4867,105.9228,darhan,MN
178,Garden Court East London,-33.0153,27.9116,east london,ZA
189,Pagėgė B&B,55.1381,21.9089,pagegiai,LT
217,Санаторий Хазино,55.8014,54.5372,kuyanovo,RU


In [52]:
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))